# Criando a base analítica para visualizações

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS analytics;
USE SCHEMA analytics;


## Base analítica — Evolução PF ao longo dos anos

In [0]:
%sql
CREATE OR REPLACE TABLE analytics.evolucao_pf_ano AS
SELECT
  ano,
  ROUND(SUM(valor_reais) / 1000000, 2) AS volume_milhoes
FROM gold.fato_transacoes_pix
WHERE pag_tipo_pessoa = 'PF'
GROUP BY ano
ORDER BY ano;


num_affected_rows,num_inserted_rows


### Gráfico de linha

In [0]:
%sql
SELECT * FROM analytics.evolucao_pf_ano;


ano,volume_milhoes
2020,68601.92
2021,2352185.59
2022,4706588.62
2023,6766699.77
2024,9484260.12
2025,10543505.25


Databricks visualization. Run in Databricks to view.

## Evolução PF → PF vs PF → PJ


Objetivo

Comparar, ao longo do tempo, o volume financeiro das transferências:

entre pessoas físicas

de pessoas físicas para pessoas jurídicas

In [0]:
%sql
CREATE OR REPLACE TABLE analytics.evolucao_pf_pf_pj AS
SELECT
  ano,
  CASE
    WHEN pag_tipo_pessoa = 'PF' AND rec_tipo_pessoa = 'PF' THEN 'PF → PF'
    WHEN pag_tipo_pessoa = 'PF' AND rec_tipo_pessoa = 'PJ' THEN 'PF → PJ'
  END AS tipo_transferencia,
  ROUND(SUM(valor_reais) / 1000000, 2) AS volume_milhoes
FROM gold.fato_transacoes_pix
WHERE pag_tipo_pessoa = 'PF'
GROUP BY ano, tipo_transferencia
ORDER BY ano, tipo_transferencia;


num_affected_rows,num_inserted_rows


### Gráfico de linha

In [0]:
%sql
SELECT * FROM analytics.evolucao_pf_pf_pj;


ano,tipo_transferencia,volume_milhoes
2020,PF → PF,60869.59
2020,PF → PJ,7732.33
2021,PF → PF,1896779.05
2021,PF → PJ,455406.53
2022,PF → PF,3706425.20
2022,PF → PJ,1000163.42
2023,PF → PF,5087800.62
2023,PF → PJ,1678899.15
2024,PF → PF,6809313.92
2024,PF → PJ,2674946.21


Databricks visualization. Run in Databricks to view.

O gráfico permite comparar a evolução do volume financeiro das transferências realizadas por pessoas físicas, evidenciando diferenças no comportamento entre transferências para outras pessoas físicas e para pessoas jurídicas ao longo do tempo.

## Volume total por região

Objetivo

Identificar quais regiões concentram maior volume financeiro de transações.

In [0]:
%sql
CREATE OR REPLACE TABLE analytics.volume_por_regiao AS
SELECT
  pag_regiao AS regiao,
  ROUND(SUM(valor_reais) / 1000000, 2) AS volume_milhoes
FROM gold.fato_transacoes_pix
GROUP BY pag_regiao
ORDER BY volume_milhoes DESC;


num_affected_rows,num_inserted_rows


### Gráfico de barras

In [0]:
%sql
SELECT * FROM analytics.volume_por_regiao;


regiao,volume_milhoes
SUDESTE,40502348.71
SUL,13015577.70
NORDESTE,11917614.95
CENTRO-OESTE,7302353.25
NORTE,4375012.21
Nao informado,67215.95


Databricks visualization. Run in Databricks to view.

O gráfico evidencia a distribuição regional do volume financeiro das transações, permitindo identificar as regiões que mais concentram envios de valores por meio do PIX.

## Valor médio por região

Objetivo

Comparar o valor médio das transações entre as regiões.

In [0]:
%sql
CREATE OR REPLACE TABLE analytics.valor_medio_por_regiao AS
SELECT
  pag_regiao AS regiao,
  ROUND(AVG(valor_reais) / 1000000, 2) AS valor_medio_milhoes
FROM gold.fato_transacoes_pix
GROUP BY pag_regiao
ORDER BY valor_medio_milhoes DESC;


num_affected_rows,num_inserted_rows


### Gráfico de barras

In [0]:
%sql
SELECT * FROM analytics.valor_medio_por_regiao;


regiao,valor_medio_milhoes
SUDESTE,373.62
SUL,129.64
NORDESTE,112.49
CENTRO-OESTE,71.96
NORTE,43.44
Nao informado,0.82


Databricks visualization. Run in Databricks to view.

O gráfico apresenta o valor médio das transações por região, evidenciando diferenças no perfil financeiro das transferências realizadas em cada localidade.

## Participação PF vs PJ

Objetivo

Avaliar a participação relativa de:

pessoas físicas

pessoas jurídicas
no volume total de transações.

In [0]:
%sql
CREATE OR REPLACE TABLE analytics.participacao_pf_pj AS
SELECT
  pag_tipo_pessoa AS tipo_pessoa,
  ROUND(SUM(valor_reais) / 1000000, 2) AS volume_milhoes
FROM gold.fato_transacoes_pix
GROUP BY pag_tipo_pessoa;


num_affected_rows,num_inserted_rows


### Gráfico 

In [0]:
%sql
SELECT * FROM analytics.participacao_pf_pj;


tipo_pessoa,volume_milhoes
PJ,43255116.71
Nao disponivel,3164.77
PF,33921841.28


Databricks visualization. Run in Databricks to view.

O gráfico apresenta a participação relativa de pessoas físicas e jurídicas no volume financeiro total das transações, evidenciando o papel de cada perfil no uso do sistema PIX.